In [78]:
from pandas import read_csv
#Load data set
look_back = 2
split = 735
series_influ_A_df = read_csv('../temp_data/influA_vietnam_last_10_days.csv', index_col=0, engine='python')
train_df = series_influ_A_df.iloc[0:split]
test_df = series_influ_A_df.iloc[split-look_back:]
# series_influ_A_df = series_influ_A_df["case"]

In [79]:
train_df.corr()

,Influenza A - All types of surveillance,temp,humidity,dew,precipcover,tempmax,tempmin,windspeed
Influenza A - All types of surveillance,1.000000,0.233416,0.016145,0.216907,0.019588,0.227861,0.231337,0.130241
temp,0.233416,1.000000,0.156404,0.950239,0.392861,0.992806,0.990106,0.269600
humidity,0.016145,0.156404,1.000000,0.454868,0.418620,0.088874,0.249285,0.122371
dew,0.216907,0.950239,0.454868,1.000000,0.487902,0.921801,0.971545,0.286277
precipcover,0.019588,0.392861,0.418620,0.487902,1.000000,0.359425,0.434643,0.141203
tempmax,0.227861,0.992806,0.088874,0.921801,0.359425,1.000000,0.968351,0.261132
tempmin,0.231337,0.990106,0.249285,0.971545,0.434643,0.968351,1.000000,0.281380
windspeed,0.130241,0.269600,0.122371,0.286277,0.141203,0.261132,0.281380,1.000000


In [80]:
series_influ_A_df.isna().sum()

Influenza A - All types of surveillance    1
temp                                       0
humidity                                   0
dew                                        0
precipcover                                0
tempmax                                    0
tempmin                                    0
windspeed                                  0
dtype: int64

In [81]:
series_influ_A_df = series_influ_A_df.rename(columns= {"Influenza A - All types of surveillance": "case"})
series_influ_A_df = series_influ_A_df[["case", "temp", "tempmax", "dew","windspeed"]]

In [82]:
def exponential_moving_average(data, span):
    return data.ewm(span=span, adjust=False).mean()

In [83]:
series_influ_A_df = series_influ_A_df.dropna()
span = 52  # Bạn có thể điều chỉnh độ dài span tùy ý
series_influ_A_df['case'] = exponential_moving_average(series_influ_A_df['case'], span)
series_influ_A_df = series_influ_A_df.astype('float32')
series_influ_A_df = series_influ_A_df.values
# normalize the dataset
from sklearn.preprocessing import MinMaxScaler, RobustScaler
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(series_influ_A_df)
# scaler = RobustScaler()
# dataset = scaler.fit_transform(series_influ_A_df)
# Create Training and Test

train = dataset[:split, :]
test = dataset[split-look_back:, :]

import numpy as np
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)

In [84]:
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [85]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import tensorflow as tf

from keras.losses import mean_squared_error

def root_mean_squared_error(y_true, y_pred):
    return tf.sqrt(mean_squared_error(y_true, y_pred))
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(5, input_shape=(2, 5)))
model.add(Dense(5))
model.compile(loss=root_mean_squared_error, optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

Epoch 1/100


c:\Users\nghiemxuan\Desktop\nghiem\GR3\GR3\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


733/733 - 2s - 2ms/step - loss: 0.2011
Epoch 2/100
733/733 - 1s - 1ms/step - loss: 0.1159
Epoch 3/100
733/733 - 1s - 925us/step - loss: 0.1044
Epoch 4/100
733/733 - 1s - 905us/step - loss: 0.0991
Epoch 5/100
733/733 - 1s - 938us/step - loss: 0.0958
Epoch 6/100
733/733 - 1s - 937us/step - loss: 0.0935
Epoch 7/100
733/733 - 1s - 1ms/step - loss: 0.0921
Epoch 8/100
733/733 - 1s - 960us/step - loss: 0.0911
Epoch 9/100
733/733 - 1s - 855us/step - loss: 0.0899
Epoch 10/100
733/733 - 1s - 1ms/step - loss: 0.0895
Epoch 11/100
733/733 - 1s - 941us/step - loss: 0.0885
Epoch 12/100
733/733 - 1s - 839us/step - loss: 0.0884
Epoch 13/100
733/733 - 1s - 822us/step - loss: 0.0876
Epoch 14/100
733/733 - 1s - 811us/step - loss: 0.0872
Epoch 15/100
733/733 - 1s - 814us/step - loss: 0.0871
Epoch 16/100
733/733 - 1s - 848us/step - loss: 0.0865
Epoch 17/100
733/733 - 1s - 867us/step - loss: 0.0866
Epoch 18/100
733/733 - 1s - 781us/step - loss: 0.0858
Epoch 19/100
733/733 - 1s - 818us/step - loss: 0.0861
Epo

In [86]:
model.save("../model/LSTM_4_4_ema.keras")